### The Zoo Dataset

In [108]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_eaf8aafe9737438a80ff8c1f0c54f1e4.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [109]:
df.shape

(101, 18)

In [110]:
df=df.rename(columns={0:"Name",17:"Type"})

In [111]:
y=df[["Type"]]
y.groupby("Type")["Type"].count()

Type
1    41
2    20
3     5
4    13
5     4
6     8
7    10
Name: Type, dtype: int64

In [112]:
X=df.drop(["Name","Type"],axis=1)
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1


In [113]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=mutual_info_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
2,0.895287
12,0.871514
9,0.701386
7,0.665912
3,0.657695
8,0.650717
0,0.555832
13,0.536032
1,0.495651
5,0.408163


In [114]:
X_scores=scores[scores["scores"]>0.5]
len(X_scores)

8

In [115]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,3,13,10,8,4,9,1,14
0,0,4,1,1,1,1,1,0
1,0,4,1,1,1,1,1,1
2,1,0,0,1,0,1,0,1
3,0,4,1,1,1,1,1,0
4,0,4,1,1,1,1,1,1


In [116]:
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [117]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
model1=OneVsRestClassifier(RandomForestClassifier(random_state=0))
scores=cross_val_score(model1,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9303846153846154 +/- 0.05218736159319775


In [11]:
y_pred = cross_val_predict(model1, X,np.asarray(y).reshape(y.shape[0],), cv=4)
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[41  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  1  2  1  1  0  0]
 [ 0  0  0 13  0  0  0]
 [ 0  0  0  0  4  0  0]
 [ 0  0  0  0  0  8  0]
 [ 0  0  0  0  0  3  7]]


In [118]:
from sklearn.multiclass import OneVsOneClassifier
model2=OneVsOneClassifier(RandomForestClassifier(random_state=0))
scores=cross_val_score(model2,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9303846153846154 +/- 0.05218736159319775


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
model3=OneVsRestClassifier(GradientBoostingClassifier(random_state=0))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9103846153846153 +/- 0.033747671407725124


In [14]:
model4=OneVsOneClassifier(GradientBoostingClassifier(random_state=0))
scores=cross_val_score(model4,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9003846153846153 +/- 0.04523933723139738


In [15]:
from sklearn import svm
model5=OneVsRestClassifier(svm.SVC(C=6,random_state=0))
scores=cross_val_score(model5,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9403846153846154 +/- 0.03486873101170823


In [16]:
model6=OneVsOneClassifier(svm.SVC(C=6,random_state=0))
scores=cross_val_score(model6,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9503846153846154 +/- 0.043682217144675205


#### The following is an experiment to examine the impact of the number of neighbors in KNN on sparse data.

In [119]:
from sklearn.neighbors import KNeighborsClassifier
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=1))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9403846153846154 +/- 0.02039187098740238


In [120]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=2))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9307692307692308 +/- 0.032960438900966976


In [121]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=3))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9207692307692308 +/- 0.028315634337393772


In [122]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=4))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9011538461538461 +/- 0.034027051542653376


In [123]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8815384615384615 +/- 0.04779146020618174


In [124]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=6))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8415384615384616 +/- 0.028409515865568277


In [125]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=7))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7919230769230771 +/- 0.07176040839500392


In [126]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=8))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.7815384615384616 +/- 0.06776220167860071


In [127]:
model7=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=9))
scores=cross_val_score(model7,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.761923076923077 +/- 0.075927207431871


In [135]:
KNN1=KNeighborsClassifier(n_neighbors=1)
scores=cross_val_score(KNN1,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9403846153846154 +/- 0.02039187098740238


In [136]:
KNN2=KNeighborsClassifier(n_neighbors=2)
scores=cross_val_score(KNN2,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9407692307692308 +/- 0.034219963722361864


In [137]:
KNN3=KNeighborsClassifier(n_neighbors=3)
scores=cross_val_score(KNN3,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9111538461538461 +/- 0.03216773715400987


In [138]:
KNN3=KNeighborsClassifier(n_neighbors=4)
scores=cross_val_score(KNN3,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8811538461538461 +/- 0.001998520162579457


In [139]:
KNN3=KNeighborsClassifier(n_neighbors=5)
scores=cross_val_score(KNN3,Xp,np.asarray(y).reshape(y.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8711538461538461 +/- 0.01808510453155193


#### The most accurate and most stable model was obtained when using only 2 neighbors. As the number of nearest neighbors increased, the model's accuray decreased. Also, it appeared that wrapping KNeighborsClassifier inside OneVsRestClassifier had no impact on the model's perofrmance when K=1. However, from 2 on, the impact of OneVsRestClassifier began to show. When K=2, although the wrapping did not improve accuracy, it produced a slightly lower standard deviation. For K=3, 4, 5, the wrapping improved accuracy by 1 to 2%.

### The Glass Dataset

In [83]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_951b21e87e79400f924e0a36d3909e64.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [84]:
df.shape

(214, 11)

In [85]:
df=df.rename(columns={10:"Class"})
y=df[["Class"]]
y.groupby("Class")["Class"].count()

Class
1    70
2    76
3    17
5    13
6     9
7    29
Name: Class, dtype: int64

In [86]:
X=df.drop(["Class"],axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0


In [87]:
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
0,1.484166
4,0.413749
3,0.400622
1,0.361431
6,0.313747
2,0.294428
7,0.247263
8,0.218857
5,0.097811
9,0.062361


In [88]:
X_scores=scores[scores["scores"]>0.3]
len(X_scores)

5

In [89]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,0,4,3,1,6
0,1,1.10,4.49,1.52101,0.06
1,2,1.36,3.60,1.51761,0.48
2,3,1.54,3.55,1.51618,0.39
3,4,1.29,3.69,1.51766,0.57
4,5,1.24,3.62,1.51742,0.55


In [90]:
model1=OneVsRestClassifier(RandomForestClassifier(random_state=0))
scores=cross_val_score(model1,Xp,np.asarray(y).reshape(y.shape[0],),cv=11)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=11.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9531100478468898 +/- 0.08935975142529189


In [91]:
model2=OneVsOneClassifier(RandomForestClassifier(random_state=0))
scores=cross_val_score(model2,Xp,np.asarray(y).reshape(y.shape[0],),cv=11)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=11.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9576555023923444 +/- 0.09653043346561677


#### OneVsRest produced better result than OneVsOne.

#### The following is an experiment conducted to examine the impact of the number of nearst neighbors in KNN on sparse data.

In [92]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=1))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9720634920634921 +/- 0.07646511348482334


In [93]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=2))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9673015873015873 +/- 0.09284072572278657


In [94]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=3))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9720634920634921 +/- 0.07646511348482334


In [95]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=4))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9577777777777778 +/- 0.10948745658479747


In [96]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9580952380952381 +/- 0.09876332827187359


In [97]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=6))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9530158730158731 +/- 0.1264297422199897


In [98]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=7))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9492063492063492 +/- 0.11692283721081204


In [99]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=8))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9482539682539683 +/- 0.14358404131775518


In [100]:
model3=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=9))
scores=cross_val_score(model3,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy:  0.9441269841269843 +/- 0.13647618678393272


#### The above was an experiment conducted to show the effect of the number of nearest neighbors in sparse data. As shown, lower K values produced more accurate and more stable models.

### The two experiments above show that for sparse data, using a lower number or K delivered better results.

In [101]:
KNN=KNeighborsClassifier(n_neighbors=1)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9720634920634921 +/- 0.07646511348482334


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [102]:
KNN=KNeighborsClassifier(n_neighbors=2)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9679365079365081 +/- 0.08230369194168051


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [103]:
KNN=KNeighborsClassifier(n_neighbors=3)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9720634920634921 +/- 0.07646511348482334


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [104]:
KNN=KNeighborsClassifier(n_neighbors=4)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9546031746031747 +/- 0.12457476169836562


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [105]:
KNN=KNeighborsClassifier(n_neighbors=5)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9580952380952381 +/- 0.09876332827187359


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [106]:
KNN=KNeighborsClassifier(n_neighbors=6)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9498412698412698 +/- 0.13207875369691569


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


In [107]:
KNN=KNeighborsClassifier(n_neighbors=7)
scores=cross_val_score(KNN,Xp,np.asarray(y).reshape(y.shape[0],),cv=15)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9492063492063492 +/- 0.11692283721081204


/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=15.
  % (min_groups, self.n_splits)), UserWarning)


### For the "glass" dataset, wrapping KNeighborsClassifier inside KNN seemed to have no impact on the model's performance. But a closer look showed that for some K values, it still either improved accuracy or made the model more stable.